# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import g_key


ModuleNotFoundError: No module named 'config'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [65]:
weather_df = pd.read_csv("City_Data.csv", encoding="utf-8")
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Humidity,Max Temp,Cloudiness,Wind Speed,Date
0,0,okhotsk,59.38,143.30,66,60.71,100,18.99,1597886060
1,1,ilulissat,69.22,-51.10,43,51.80,0,5.82,1597886097
2,2,hobart,-42.88,147.33,50,54.00,40,18.34,1597885914
3,3,victoria,22.29,114.16,79,84.99,68,8.99,1597885828
4,4,manono,-7.30,27.42,36,71.01,54,2.42,1597886355


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [66]:
humidity = weather_df["Humidity"]
maxhumidity = humidity.max()
locations = weather_df[["Latitude", "Longitude"]]
locations

,Latitude,Longitude
0,59.38,143.30
1,69.22,-51.10
2,-42.88,147.33
3,22.29,114.16
4,-7.30,27.42
...,...,...
551,44.56,-123.26
552,45.52,-122.68
553,46.20,-59.96
554,51.03,-4.22


In [67]:
figure = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=2)
figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
ideal_df = weather_df[(weather_df['Max Temp'].between(75,92)) & (weather_df['Wind Speed']<=8) & (weather_df['Humidity']<=45)]
ideal_df.reset_index(inplace=True)
del ideal_df['index']


ideal_df


,Unnamed: 0,City,Latitude,Longitude,Humidity,Max Temp,Cloudiness,Wind Speed,Date
0,23,marzuq,14.40,46.47,44,80.26,27,6.98,1597886357
1,37,makakilo city,21.35,-158.09,45,89.60,75,5.82,1597886060
2,98,diamantino,-14.41,-56.45,22,83.70,0,2.55,1597886126
3,159,prescott valley,34.59,-112.33,22,89.60,1,4.70,1597886271
4,178,orizona,-17.03,-48.30,31,79.23,100,4.72,1597886382
5,214,ramshir,30.89,49.41,37,82.40,0,4.70,1597886390
6,218,riyadh,24.69,46.72,22,80.60,100,2.24,1597886074
7,220,kashan,33.98,51.44,21,78.71,0,6.89,1597886391
8,304,coxim,-18.51,-54.76,29,80.96,24,4.16,1597886407
9,312,arlit,18.74,7.39,12,91.90,0,5.57,1597886409


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
hotel_df = []

for x in range(len(ideal_df)):
    lat = ideal_df.loc[x]['Latitude']
    lng = ideal_df.loc[x]['Longitude']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    
    try:
        hotel_df.append(response['results'][0]['name'])
    except:
        hotel_df.append("")
        


In [76]:
ideal_df["Hotel Name"] = hotel_df
ideal_df
ideal_df = ideal_df.dropna(how='any')
ideal_df

,Unnamed: 0,City,Latitude,Longitude,Humidity,Max Temp,Cloudiness,Wind Speed,Date,Hotel Name
0,23,marzuq,14.40,46.47,44,80.26,27,6.98,1597886357,Al Hujayrah
1,37,makakilo city,21.35,-158.09,45,89.60,75,5.82,1597886060,Kapolei
2,98,diamantino,-14.41,-56.45,22,83.70,0,2.55,1597886126,Diamantino
3,159,prescott valley,34.59,-112.33,22,89.60,1,4.70,1597886271,Prescott
4,178,orizona,-17.03,-48.30,31,79.23,100,4.72,1597886382,Orizona
5,214,ramshir,30.89,49.41,37,82.40,0,4.70,1597886390,Ramshir
6,218,riyadh,24.69,46.72,22,80.60,100,2.24,1597886074,Riyadh
7,220,kashan,33.98,51.44,21,78.71,0,6.89,1597886391,Kashan
8,304,coxim,-18.51,-54.76,29,80.96,24,4.16,1597886407,Coxim
9,312,arlit,18.74,7.39,12,91.90,0,5.57,1597886409,Arlit


In [80]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Latitude", "Longitude"]]
locations

,Latitude,Longitude
0,14.40,46.47
1,21.35,-158.09
2,-14.41,-56.45
3,34.59,-112.33
4,-17.03,-48.30
5,30.89,49.41
6,24.69,46.72
7,33.98,51.44
8,-18.51,-54.76
9,18.74,7.39


In [81]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))